In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy (for lemmatization only)
import spacy

# plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/Users/jiong/opt/miniconda3/envs/py38-nlp/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [31]:
# Prepare Stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['add', 'still', 'amp', 'inflation'])

In [32]:
df = pd.read_csv('cleaned_lemmatized_tweets_data.csv')
df.head()

,Unnamed: 0,id,date,tweet,hashtags,username,place,geo,timezone,parse_tweet
0,0,1353492222344392707,2021-01-25 07:57:47,idk bothering alcohol inflation time high rn,[],MphoKgosidialwa,NaN,NaN,800,idk bother alcohol inflation time high rn
1,1,1353492195827965952,2021-01-25 07:57:40,inflation much money means prices rise eventua...,[],AllenWi92526840,NaN,NaN,800,inflation much money mean price rise eventuall...
2,2,1353492090311720960,2021-01-25 07:57:15,exerts daily fix talk inflation roll deafening...,"['fomc', 'trading']",ChrisWeston_PS,NaN,NaN,800,exert daily fix talk inflation roll deafen exu...
3,3,1353492059370479617,2021-01-25 07:57:08,regards michellins explains recent price rises...,[],morejunkfromu,NaN,NaN,800,regard michellin explain recent price rise inf...
4,4,1353492005008113665,2021-01-25 07:56:55,oh mike since clearly unaware inflation levels...,[],apecapital,NaN,NaN,800,oh mike since clearly unaware inflation level ...


In [33]:
data = df.parse_tweet.values.tolist()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) # deacc=True removes punctuations (deaccent)

data_words = list(sent_to_words(data))

In [34]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [35]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[doc] for doc in texts]

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [36]:
# Remove Stop Words
data_words_nostopwords = remove_stopwords(data_words)

# Create Bigrams
bigrams_data = make_bigrams(data_words_nostopwords)

trigram_data = make_trigrams(data_words_nostopwords)

In [37]:
def lda_builder(n_gram):
# Create Dictionary
    id2word = corpora.Dictionary(n_gram)

    # Create Corpus
    texts = n_gram

    # Create Term Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    # Building the Topic Model using LDA - fit the model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, random_state=100, update_every=1, num_topics=6, chunksize=100,passes=10, alpha='auto',per_word_topics=True)#num_topics=5
    return lda_model, corpus, id2word

In [38]:
bigram_model, bigram_corpus, bigram_id2word  = lda_builder(bigrams_data)
trigram_model, trigram_corpus, trigram_id2word = lda_builder(trigram_data)

In [39]:
import warnings
warnings.filterwarnings('ignore')
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(trigram_model, trigram_corpus, trigram_id2word, mds='mmds')
vis

/Users/jiong/opt/miniconda3/envs/py38-nlp/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jiong/opt/miniconda3/envs/py38-nlp/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jiong/opt/miniconda3/envs/py38-nlp/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jiong/opt/miniconda3/envs/py38-nlp/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jiong/opt

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.044862  0.263585       1        1  24.325074
2     -0.101314 -0.281142       2        1  20.421628
1     -0.253336  0.053815       3        1  15.349660
4      0.191868  0.133925       4        1  14.436792
3      0.235528 -0.125062       5        1  13.627891
0     -0.027882 -0.045120       6        1  11.838955, topic_info=         Term       Freq      Total Category  logprob  loglift
269    dollar  52.000000  52.000000  Default  30.0000  30.0000
203      year  58.000000  58.000000  Default  29.0000  29.0000
394      back  44.000000  44.000000  Default  28.0000  28.0000
12      price  60.000000  60.000000  Default  27.0000  27.0000
215  increase  37.000000  37.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
888     print   3.514174   9.046660   Topic6  -5.3971   1.1882
350      look   3.309034  10.565286   Topic6  -5.4572   0.9729
29        see   3.331201  15.352929   Topic6  -5.4506   0.6058
447       way   3.058417  13.635465   Topic6  -5.5360   0.6390
341       tax   2.868698  12.370851   Topic6  -5.6000   0.6723

[250 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
323       1  0.906126  adjust
1219      2  0.975728  afford
805       1  0.956472     ago
1182      2  0.962884   agree
2514      4  0.943309    ally
...     ...       ...     ...
94        3  0.950845   world
1122      3  0.982839   worth
203       1  0.945175    year
203       2  0.034370    year
2511      5  0.963536  yellow

[270 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 2, 5, 4, 1])